In [1]:
# univariate multi-step encoder-decoder lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [18]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [23]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90,120,130, 140, 160,130, 120, 100, 80, 60, 30,10, 30,60, 90, 120,90, 80]

# choose a number of time steps
n_steps_in, n_steps_out = 4, 3

# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))



In [24]:
print(X)

[[[ 10]
  [ 20]
  [ 30]
  [ 40]]

 [[ 20]
  [ 30]
  [ 40]
  [ 50]]

 [[ 30]
  [ 40]
  [ 50]
  [ 60]]

 [[ 40]
  [ 50]
  [ 60]
  [ 70]]

 [[ 50]
  [ 60]
  [ 70]
  [ 80]]

 [[ 60]
  [ 70]
  [ 80]
  [ 90]]

 [[ 70]
  [ 80]
  [ 90]
  [120]]

 [[ 80]
  [ 90]
  [120]
  [130]]

 [[ 90]
  [120]
  [130]
  [140]]

 [[120]
  [130]
  [140]
  [160]]

 [[130]
  [140]
  [160]
  [130]]

 [[140]
  [160]
  [130]
  [120]]

 [[160]
  [130]
  [120]
  [100]]

 [[130]
  [120]
  [100]
  [ 80]]

 [[120]
  [100]
  [ 80]
  [ 60]]

 [[100]
  [ 80]
  [ 60]
  [ 30]]

 [[ 80]
  [ 60]
  [ 30]
  [ 10]]

 [[ 60]
  [ 30]
  [ 10]
  [ 30]]

 [[ 30]
  [ 10]
  [ 30]
  [ 60]]

 [[ 10]
  [ 30]
  [ 60]
  [ 90]]]


In [25]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=100, verbose=0)

# demonstrate prediction
x_input = array([70, 80, 90, 100])

In [26]:
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[120.03701]
  [136.92834]
  [138.29527]]]
